# Individual Assignment 2 - Moritz Voss 482724

## Import libraries

In [3]:
import pandas as pd
import gurobipy as gp
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import plotly.express as px

## Q1

In [2]:
dic_swim1 = {('Nadine', 'BA'): 40.1, ('Nadine', 'BR'): 41.7, ('Nadine', 'BU'): 41.1, ('Nadine', 'FR'): 33.3, ('Mary', 'BA'): 39.6, ('Mary', 'BR'): 37.6, ('Mary', 'BU'): 31.9, 
            ('Mary', 'FR'): 31.2, ('Lina', 'BA'): 35.3, ('Lina', 'BR'): 37.9, ('Lina', 'BU'): 36.2, ('Lina', 'FR'): 33.1, ('Asma', 'BA'): 30.6, ('Asma', 'BR'): 34.1, 
            ('Asma', 'BU'): 33.3, ('Asma', 'FR'): 29.1, ('Carol', 'BA'): 36.9, ('Carol', 'BR'): 37.2, ('Carol', 'BU'): 29.0, ('Carol', 'FR'): 31.6}

names = ['Nadine', 'Mary', 'Lina', 'Asma', 'Carol']
styles = ['BA', 'BR', 'BU', 'FR']

model = gp.Model()
    
# X mdoels all permutation 
x = model.addVars(dic_swim1, obj = dic_swim1, vtype=gp.GRB.BINARY, name = ' ')
model.ModelSense = gp.GRB.MINIMIZE

# Constarint
for i in names:
    model.addConstr(gp.quicksum((x[(i, j)] for j in styles)) <= 1)
    
for j in styles:
    model.addConstr(gp.quicksum((x[(i, j)] for i in names)) == 1)
    
model.optimize()
model.printAttr('X')
print (model.ObjVal)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-03
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x69b34e51
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 136.3000000
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 40 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 135.8000000

Root relaxation: objective 1.287000e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It

## Q2a

In [3]:
n = 10000
max = 100
booking_limit = list(range(20,81))
customerP1 = np.random.poisson(lam=75,size=10000)
customerP2 = np.random.poisson(lam=50,size=10000)
profitAlpha = []

for i in booking_limit:
    salesP1 = np.where(customerP1 <= i, customerP1, i)
    ticketsLeft = max - salesP1
    salesP2 = np.where(customerP2 > ticketsLeft, ticketsLeft, customerP2)
    revenue = (150*salesP1 + 500*salesP2)
    profitAlpha.append([i, np.mean(revenue)])

In [4]:
profitAlpha = pd.DataFrame(profitAlpha, columns=['Alpha', 'Expected Revenue'])
profitAlpha.sort_values('Expected Revenue')

,Alpha,Expected Revenue
60,80,24229.315
59,79,24333.615
58,78,24452.475
57,77,24586.945
56,76,24736.080
...,...,...
24,44,31239.250
28,48,31255.505
25,45,31276.750
27,47,31286.320


In [5]:
px.line(profitAlpha, x = 'Alpha', y = 'Expected Revenue')

## Q2b

In [6]:
n = 10000
max = 100
booking_limit = list(range(20,81))

compensation = list(range(100,301))

customerP1 = np.random.poisson(lam=75,size=10000)
customerP2 = np.random.poisson(lam=50,size=10000)
profitAlpha2 = []

for i in booking_limit:
    for p in compensation:
        salesP1 = np.where(customerP1 <= i, customerP1, i)
        
        prob = np.random.binomial(salesP1, ((p-100)/200), 10000)
        
        ticketsLeft = max - salesP1
        
        p2_shoratge = np.where(ticketsLeft < customerP2, customerP2 - ticketsLeft, 0)
        p1_refunds = np.where(prob >= p2_shoratge, p2_shoratge, prob)
        
        costsRefund = p1_refunds * p
        
        newCustomersP1 = salesP1 - p1_refunds
        ticketsLeft += p1_refunds
        
        salesP2 = np.where(customerP2 > ticketsLeft, ticketsLeft, customerP2)
        
        revenue = (150 * newCustomersP1 + 500 * salesP2 - costsRefund)
        profitAlpha2.append([i, p, np.mean(revenue)])

In [7]:
profitAlpha2 = pd.DataFrame(profitAlpha2, columns=['Alpha', 'Refund Tickets', 'Expected Revenue'])
profitAlpha2.sort_values('Expected Revenue')

,Alpha,Refund Tickets,Expected Revenue
12060,80,100,24265.3850
12061,80,101,24355.3985
11859,79,100,24370.4200
12062,80,102,24446.7474
11860,79,101,24456.8479
...,...,...,...
6082,50,152,31606.2126
6080,50,150,31606.8950
6078,50,148,31607.1412
6079,50,149,31608.3873


In [8]:
import plotly.graph_objects as go

# get the max revenue per alpha
df_max = profitAlpha2.loc[profitAlpha2.groupby(["Alpha"])["Expected Revenue"].idxmax()] 

trace1  = go.Scatter(
        mode='lines+markers',
        x = df_max['Alpha'],
        y = df_max['Expected Revenue'],
        name="Expected Revenue"
    )

trac3  = go.Scatter(
        mode='lines+markers',
        x = profitAlpha['Alpha'],
        y = profitAlpha['Expected Revenue'],
        name="Expected Revenue Old"
    )

trace2 = go.Bar(
        x = df_max['Alpha'],
        y = df_max['Refund Tickets'],
        name="Refund Tickets",
        yaxis='y2',
        opacity=0.5
    )

data = [trace1, trace2, trac3]

layout = go.Layout(
    title_text='Seat Simulaton with Callable Products',
    template='ggplot2',
    yaxis=dict(
        side = 'right',
        showgrid = False
    ),
    yaxis2=dict(
        overlaying='y',
        anchor='y3',
        showgrid = False
    ),
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
    )
)
fig = go.Figure(data=data, layout=layout)
fig.show()

# fig = px.line(df_max, x = 'Alpha', y = 'Expected Revenue')
# fig.show()

## Q3a

In [1]:
# # Setting up variables
# cost = 10
# profit = 20

# warehouse = ['A', 'B']
# distributionCenter = [1, 2, 3, 4]

# scenarios = ['a', 'b', 'c']
# scenorioProbs = {'a': 0.3, 'b': 0.2, 'c': 0.5}

# transportationCosts = {('A', 1): 1.5, ('A', 2): 2, ('A', 3): 2, ('A', 4): 3, ('B', 1): 2, ('B', 2): 1.2, ('B', 3): 1.1, ('B', 4): 2.5}
# demands = {('a', 1): 5, ('a', 2): 3, ('a', 3): 7.8, ('a', 4): 4, ('b', 1): 3, ('b', 2): 1, ('b', 3): 6, ('b', 4): 3, ('c', 1): 4.2, ('c', 2): 4.1, ('c', 3): 3, ('c', 4): 5.4}

# PF = {'A': 1, 'B': 1.5}


# # Gurobi model
# model = gp.Model()

# # First stage variable
# ## One for production
# products = model.addVars(warehouse, vtype=gp.GRB.INTEGER, name = ' ')
# ## One for shipping
# ship = model.addVars(warehouse, distributionCenter, vtype=gp.GRB.INTEGER, name = ' ')

# # Second stage variable
# ## One for demand/sold
# sold = model.addVars(demands, vtype=gp.GRB.INTEGER, name = ' ')

# # Objective 
# obj = 0
# obj -= gp.quicksum(products[w]*cost for w in warehouse) # cost of production
# obj -= gp.quicksum(products[w]*PF[w] for w in warehouse) # costs of shipping to warehouse
# obj -= gp.quicksum(ship[w, d]*transportationCosts[(w, d)] for w in warehouse for d in distributionCenter) # transportation costs to distribution centers
# obj += gp.quicksum(sold[s, d]*profit*scenorioProbs[p] for s in scenarios for d in distributionCenter for p in scenorioProbs) # TODO: need a variable for sold - not correct YET!

# model.setObjective(obj, gp.GRB.MAXIMIZE)

# # Constraints

# # We can not sell more than the demand
# model.addConstrs(sold[s, d] <= demands[s, d] for d in distributionCenter for s in scenarios)

# # We cannot sell more then we shiped
# model.addConstrs(sold[s, d]  <= ship[w, d] for w in warehouse for d in distributionCenter for s in scenarios)

# # We cannot ship more from a PF then we produced
# model.addConstrs(gp.quicksum(ship[w, d] for d in distributionCenter) <=  products[w] for w in warehouse)

# model.optimize()
# model.printAttr(["X"])
# # print('Total cost:', model.OBJVAL)

In [12]:
# Setting up variables
cost = 10
profit = 20

warehouses = ['A', 'B']
warehouse_cost = {'A': 1, 'B': 1.5}

distribution_centers = [1, 2, 3, 4]
distribution_cost = {('A', 1): 1.5, ('A', 2): 2, ('A', 3): 2, ('A', 4): 3, ('B', 1): 2, ('B', 2): 1.2, ('B', 3): 1.1, ('B', 4): 2.5}

scenarios = ['a', 'b', 'c']
senario_probabilities = {'a': 0.3, 'b': 0.2, 'c': 0.5}

# demands = {('a', 1): 5, ('a', 2): 3, ('a', 3): 7.8, ('a', 4): 4, ('b', 1): 3, ('b', 2): 1, ('b', 3): 6, ('b', 4): 3, ('c', 1): 4.2, ('c', 2): 4.1, ('c', 3): 3, ('c', 4): 5.4}

demands = {('a', 1): 5.0*1000, ('a', 2): 3.0*1000, ('a', 3): 7.8*1000, ('a', 4): 4.0*1000, 
           ('b', 1): 3.0*1000, ('b', 2): 1.0*1000, ('b', 3): 6.0*1000, ('b', 4): 3.0*1000, 
           ('c', 1): 4.2*1000, ('c', 2): 4.1*1000, ('c', 3): 3.0*1000, ('c', 4): 5.4*1000}

model = gp.Model('Transportation')

# First
to_produce = model.addVar(name='product', vtype=gp.GRB.INTEGER)
PtW = model.addVars(warehouses, vtype = gp.GRB.INTEGER, name = 'PtW')

# Second
WtD = model.addVars(warehouses, distribution_centers, scenarios, vtype = gp.GRB.INTEGER, name = 'WtD')
SE = model.addVars(distribution_centers, scenarios, vtype = gp.GRB.INTEGER, name = 'SE')

# Set Objective
obj = 0
obj -= to_produce * cost # cost to produce
obj -= gp.quicksum(PtW[w] * warehouse_cost[w] for w in warehouse_cost) # cost to ship to warehouse
obj -= gp.quicksum(senario_probabilities[s] * WtD[w,d,s] * distribution_cost[(w,d)] for w in warehouses for d in distribution_centers for s in scenarios) # cost to ship from warehouse to distribution center
obj += gp.quicksum(senario_probabilities[s] * SE[d, s] * profit for d in distribution_centers for s in scenarios) # profits from sales

model.setObjective(obj, gp.GRB.MAXIMIZE)

# Add Constraints
# Everything we produce must be sent to A or B
model.addConstr(gp.quicksum(PtW[w] for w in warehouses) == to_produce)

# # What is in the warehouse can be larger or equal to what is send out
# model.addConstrs(PtW[w] >= WtD[w, d, s] for w in warehouses for d in distribution_centers for s in scenarios)

# We can not ship more form the warehouses than we produced
model.addConstrs(gp.quicksum(WtD[w, d, s] for d in distribution_centers) <= PtW[w] for w in warehouses for s in scenarios)

# Sales can not be bigger than the demand
model.addConstrs(SE[d,s]<=demands[s,d] for d in distribution_centers for s in scenarios)

# The sales cannot be bigger then the products at the distribution center
model.addConstrs(SE[d,s] <= gp.quicksum(WtD[w, d, s] for w in warehouses) for d in distribution_centers for s in scenarios)

model.optimize()
model.printAttr(["X"])
print('Total cost:', model.OBJVAL)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 55 rows, 39 columns and 129 nonzeros
Model fingerprint: 0xe1c4a945
Variable types: 0 continuous, 39 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 37 rows and 1 columns
Presolve time: 0.00s
Presolved: 18 rows, 38 columns, 66 nonzeros
Variable types: 0 continuous, 38 integer (0 binary)

Root relaxation: objective 1.049990e+05, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    104999.00000 104999.000  0.00%     -    0s

Explored 1 

## Q3b

In [5]:
df_demand = pd.read_csv('/Users/mlvos/Desktop/Moritz/Education/Erasmus University/Management Science/Individual Assignment 2/demand.csv', encoding='latin-1')
df_transportation_cost = pd.read_csv('/Users/mlvos/Desktop/Moritz/Education/Erasmus University/Management Science/Individual Assignment 2/TranportationCosts.csv', encoding='latin-1')

In [6]:
# Setting up variables
cost = 10
profit = 20
fixed_cost = 100000

In [7]:
warehouses = list(df_transportation_cost['Warehouse'])
warehouse_cost = {}
for index, row in df_transportation_cost[['Warehouse', 'PFacility']].iterrows():
    warehouse_cost[row.Warehouse] = row.PFacility

distribution_centers = list(df_demand.DistributionCenter)
distribution_cost = {}
for wh in warehouses:
    for dc in distribution_centers:
        distribution_cost[(wh, dc)] = df_transportation_cost.set_index('Warehouse').loc[wh, dc]
        
scenarios = list(df_demand.columns.drop('DistributionCenter'))
senario_probabilities = {}
for s in scenarios:
    senario_probabilities[s] = .01
    
demands = {}
for s in scenarios:
    for dc in distribution_centers:
        demands[(s, dc)] = df_demand.set_index('DistributionCenter').loc[dc, s]

In [8]:
bigM = 300000

In [10]:
model = gp.Model('Transportation')

## Declare Variables
# First
to_produce = model.addVar(name='product', vtype=gp.GRB.INTEGER)
Wh = model.addVars(warehouses, vtype = gp.GRB.BINARY, name = 'Wh')
PtW = model.addVars(warehouses, vtype = gp.GRB.INTEGER, name = 'PtW')

# Second
WtD = model.addVars(warehouses, distribution_centers, scenarios, vtype = gp.GRB.INTEGER, name = 'WtD')
SE = model.addVars(distribution_centers, scenarios, vtype = gp.GRB.INTEGER, name = 'SE')

## Set Objective
obj = 0
obj -= to_produce * cost # cost to produce
obj -= gp.quicksum(Wh[w] * fixed_cost for w in warehouses) # fixed warehouse cost
obj -= gp.quicksum(PtW[w] * warehouse_cost[w] for w in warehouse_cost) # cost to ship to warehouse
obj -= gp.quicksum(senario_probabilities[s] * WtD[w,d,s] * distribution_cost[(w,d)] for w in warehouses for d in distribution_centers for s in scenarios) # cost to ship from warehouse to distribution center
obj += gp.quicksum(senario_probabilities[s] * SE[d, s] * profit for d in distribution_centers for s in scenarios) # profits from sales

model.setObjective(obj, gp.GRB.MAXIMIZE)

## Add Constraints
# Everything we produce must be sent to warehouse A or B
model.addConstr(gp.quicksum(PtW[w] for w in warehouses) == to_produce)

# We can not ship more from a warehouses than we send to it
model.addConstrs(gp.quicksum(WtD[w, d, s] for d in distribution_centers) <= PtW[w] for w in warehouses for s in scenarios)

# If products are send to a warehouse then the bin variable 
model.addConstrs(PtW[w] <= Wh[w] * bigM for w in warehouses)
        
# # Model if x > y, then b = 1, otherwise b = 0
# model.addConstrs(PtW[w] >= 0 + 0 - bigM * (1 - Wh[w]) for w in warehouses)
# model.addConstrs(PtW[w] <= 0 + bigM * Wh[w] for w in warehouses)

# # Add indicator constraints
# model.addConstrs((Wh[w] == 1) >> (PtW[w] == PtW[w]) for w in warehouses)
# model.addConstrs((Wh[w] == 0) >> (PtW[w] ==  0) for w in warehouses)

# Sales can not be bigger than the demand
model.addConstrs(SE[d,s] <= demands[s,d] for d in distribution_centers for s in scenarios)

# The sales cannot be bigger then the products at the distribution center
model.addConstrs(SE[d,s] <= gp.quicksum(WtD[w, d, s] for w in warehouses) for d in distribution_centers for s in scenarios)

model.optimize()
# model.printAttr(["X"])
print('Total cost:', model.OBJVAL)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10524 rows, 98447 columns and 199170 nonzeros
Model fingerprint: 0x14fda494
Variable types: 0 continuous, 98447 integer (23 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+05]
  Objective range  [9e-03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+02, 4e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 4101 rows and 1 columns
Presolve time: 0.19s
Presolved: 6423 rows, 98446 columns, 195046 nonzeros
Variable types: 0 continuous, 98446 integer (23 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 2.210736e+06, 13637 iterations, 0.89 seconds (0.80 work units)

    Nodes    |    Current Node    |     Objective Bounds      |

In [78]:
# extract optimal pairs from the gurobi model
# results = []
# for k,v in WtD.items():
#     print(k, k, v.X)

vals = {k : v.X for k,v in WtD.items()}
vals

df_vals = pd.DataFrame([k + (v,) for k, v in vals.items()], columns=['WH', 'DC', 'Scenraio', 'Sent'])
df_vals

,WH,DC,Scenraio,Sent
0,WH-Seville,DC-Vienna,S1,-0.0
1,WH-Seville,DC-Vienna,S2,-0.0
2,WH-Seville,DC-Vienna,S3,-0.0
3,WH-Seville,DC-Vienna,S4,-0.0
4,WH-Seville,DC-Vienna,S5,-0.0
...,...,...,...,...
94295,WH-Bordeaux,DC-Marseille,S96,-0.0
94296,WH-Bordeaux,DC-Marseille,S97,-0.0
94297,WH-Bordeaux,DC-Marseille,S98,-0.0
94298,WH-Bordeaux,DC-Marseille,S99,-0.0


In [82]:
df_stuttgart = df_vals.loc[df_vals['WH'] == 'WH-Stuttgart']
df_stuttgart

,WH,DC,Scenraio,Sent
12300,WH-Stuttgart,DC-Vienna,S1,8831.0
12301,WH-Stuttgart,DC-Vienna,S2,10174.0
12302,WH-Stuttgart,DC-Vienna,S3,7921.0
12303,WH-Stuttgart,DC-Vienna,S4,10678.0
12304,WH-Stuttgart,DC-Vienna,S5,13586.0
...,...,...,...,...
16395,WH-Stuttgart,DC-Marseille,S96,2579.0
16396,WH-Stuttgart,DC-Marseille,S97,2868.0
16397,WH-Stuttgart,DC-Marseille,S98,3344.0
16398,WH-Stuttgart,DC-Marseille,S99,4576.0


In [39]:
stuttagrt = []
for k, v in distribution_cost:
    stuttagrt.append(distribution_cost['WH-Stuttgart', v])
 
sum = 0   
for i in stuttagrt:
    sum += i
    
sum/len(stuttagrt)

1.5487804878048805

In [51]:
df = pd.DataFrame([k + (v,) for k, v in distribution_cost.items()], columns=['WH', 'DC', 'Cost'])
df.groupby('WH').mean().sort_values('Cost')

,Cost
WH,
WH-Stuttgart,1.548780
WH-Charleroi,1.558537
WH-Bordeaux,1.673171
WH-Bristol,1.682927
WH-KrakÃ³w,1.734146
WH-Palma de Mallorca,1.795122
WH-Glasgow,1.814634
WH-Oslo,1.846341
WH-Hanover,1.860976


In [69]:
df_demand['mean'] = df_demand.iloc[:, 1:].mean(axis=1)
df_demand['mean'].sum()


281632.8367337312